In [3]:
from model import *
from data import *
from metrics import *
import tensorflow as tf
from tensorflow.keras.backend import int_shape
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.models import Model

from tensorflow.keras import layers 
from tensorflow.keras.layers import (BatchNormalization, Conv2D, Conv2DTranspose, MaxPooling2D, Dropout, SpatialDropout2D, 
                                     UpSampling2D, Input, concatenate, multiply, add, Activation, Cropping2D)
%load_ext autoreload
%autoreload 2

In [4]:
data_gen_args = dict(rotation_range=30,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    zoom_range=[0.8,1],
                    brightness_range=[0.9,1.1],
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')

image_generator, image_val_generator, mask_generator, mask_val_generator = trainGenerator(16,'dataset','images_satellite','labels_satellite',data_gen_args)
        
train_generator = train(image_generator, mask_generator)
val_generator = val(image_val_generator, mask_val_generator)

Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.


Train form data in dataset using data augmentation

In [5]:
model = VanillaUnet()
model.compile(optimizer = Adam(lr = 1e-4), loss = [binary_focal_loss()], metrics = ['accuracy', mean_iou, mean_iou_neg, mean_iou_weighted, dice, dice_neg, dice_weighted])
model_checkpoint = ModelCheckpoint('vanilla_final.hdf5', monitor='val_mean_iou_weighted',verbose=1, save_best_only=True, mode = 'max')
history=model.fit_generator(train_generator,steps_per_epoch=400,epochs=15,validation_data=val_generator,validation_steps=50, callbacks=[model_checkpoint])

(None, 32, 32, 256)
(None, 32, 32, 512)
(None, 256, 256, 1)
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
400/400 [==============================] - ETA: 0s - loss: 0.0287 - accuracy: 0.6266 - mean_iou: 0.3012 - mean_iou_neg: 0.4925 - mean_iou_weighted: 0.4543 - dice: 0.3918 - dice_neg: 0.5914 - dice_weighted: 0.5515
Epoch 00001: val_mean_iou_weighted improved from -inf to 0.61097, saving model to vanilla_final.hdf5
400/400 [==============================] - 186s 465ms/step - loss: 0.0287 - accuracy: 0.6266 - mean_iou: 0.3012 - mean_iou_neg: 0.4925 - mean_iou_weighted: 0.4543 - dice: 0.3918 - dice_neg: 0.5914 - dice_weighted: 0.5515 - val_loss: 0.0113 - val_accuracy: 0.6956 - val_mean_iou: 0.5052 - val_mean_iou_neg: 0.6374 - val_mean_iou_weighted: 0.6110 - val_dice: 0.6290 - val_dice_neg: 0.7456 - val_dice_weighted: 0.7223
Epoch 2/15
400/400 [==============================] - ETA: 0s - loss: 0.0115 - accuracy: 0.7029 - mean_iou: 0.5626 - mean_io

Epoch 12/15
400/400 [==============================] - ETA: 0s - loss: 0.0069 - accuracy: 0.7368 - mean_iou: 0.6728 - mean_iou_neg: 0.7571 - mean_iou_weighted: 0.7402 - dice: 0.7793 - dice_neg: 0.8435 - dice_weighted: 0.8307
Epoch 00012: val_mean_iou_weighted did not improve from 0.64240
400/400 [==============================] - 178s 446ms/step - loss: 0.0069 - accuracy: 0.7368 - mean_iou: 0.6728 - mean_iou_neg: 0.7571 - mean_iou_weighted: 0.7402 - dice: 0.7793 - dice_neg: 0.8435 - dice_weighted: 0.8307 - val_loss: 0.0139 - val_accuracy: 0.6593 - val_mean_iou: 0.5127 - val_mean_iou_neg: 0.6269 - val_mean_iou_weighted: 0.6041 - val_dice: 0.6425 - val_dice_neg: 0.7417 - val_dice_weighted: 0.7218
Epoch 13/15
400/400 [==============================] - ETA: 0s - loss: 0.0058 - accuracy: 0.7407 - mean_iou: 0.7048 - mean_iou_neg: 0.7783 - mean_iou_weighted: 0.7636 - dice: 0.8071 - dice_neg: 0.8604 - dice_weighted: 0.8497
Epoch 00013: val_mean_iou_weighted did not improve from 0.64240
400/400

In [6]:
save_obj(history.history,'vanilla_history')

Test model

In [4]:
test_img, names = test('dataset/test_images')
# print(test_img.shape)

folder = './dataset/test_labels'
num_img, size = test_img.shape[0], test_img.shape[1]            

test_label = generate_labels(folder, names, num_img, size)
# print(test_label.shape)

In [8]:
model_load = load_model('vanilla_final.hdf5', compile = False, custom_objects={'binary_focal_loss_fixed':binary_focal_loss()})
model_load.compile(optimizer = Adam(lr = 1e-4), loss = [binary_focal_loss()], metrics = ['accuracy', mean_iou, mean_iou_neg, mean_iou_weighted, dice, dice_neg, dice_weighted])

score, accuracy,  iou, iou_neg, iou_weighted, dice_coeff, dice_coeff_neg, dice_coeff_weighted= model.evaluate(test_img, test_label)
print('Test score:', score)

1/1 [==============================] - 0s 1ms/step - loss: 0.0581 - accuracy: 0.8604 - mean_iou: 0.2538 - mean_iou_neg: 0.5472 - mean_iou_weighted: 0.4885 - dice: 0.3558 - dice_neg: 0.6336 - dice_weighted: 0.5781
Test score: 0.058149587363004684


In [6]:
data_gen_args = dict(rotation_range=30,
                    horizontal_flip=True,
                    vertical_flip=True)


test_gen = trainGenerator_simple(4,'dataset','test_images','test_labels', aug_dict = data_gen_args, image_color_mode = "rgb",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 1,save_to_dir = None,
                   target_size = (256,256),seed = 1)

model = load_model('vanilla_final.hdf5',  compile = False, custom_objects={'binary_focal_loss_fixed':binary_focal_loss()})
model.compile(optimizer = Adam(lr = 1e-4), loss = [binary_focal_loss()], metrics = ['accuracy', mean_iou, mean_iou_neg, mean_iou_weighted, dice, dice_neg, dice_weighted])

score, accuracy,  iou, iou_neg, iou_weighted, dice_coeff, dice_coeff_neg, dice_coeff_weighted= model.evaluate(test_gen,steps=20)

Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
20/20 [==============================] - 2s 89ms/step - loss: 0.0574 - accuracy: 0.8619 - mean_iou: 0.1938 - mean_iou_neg: 0.5213 - mean_iou_weighted: 0.4558 - dice: 0.2804 - dice_neg: 0.5986 - dice_weighted: 0.5350
